In [0]:
!pip install -U -q PyDrive

    100% |████████████████████████████████| 993kB 17.4MB/s 


In [0]:
import glob
import pickle
import random

import numpy as np
from keras import Input, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import ReLU, Conv2D, Add, MaxPooling2D, AveragePooling2D, Flatten, Dense, BatchNormalization
from keras.optimizers import Adam
from keras.utils import Sequence

import zipfile
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files

In [0]:
data_set_id = "1iBT5SDpPLU2F5NyPiktpK3r5bAxGdi92"

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id': data_set_id})
downloaded.GetContentFile("data_set.zip")

zip_ref = zipfile.ZipFile("data_set.zip", 'r')
zip_ref.extractall("data_set")
zip_ref.close()

In [0]:
class ModelBuilder:

    def createClassifier(self, shape, n_filters=64, embedded_model=None):
        input_layer = Input(shape=shape)

        x = Conv2D(n_filters, kernel_size=(7, 7), strides=2, padding="same", activation="relu")(input_layer)
        x = MaxPooling2D(pool_size=2, padding="same")(x)

        if embedded_model is None:
            embedded_model = self.createResBlocks(shape=(shape[0] / 4, shape[1] / 4, n_filters), n_filters=n_filters)
        x = embedded_model(x)

        x = AveragePooling2D(pool_size=2, padding="same")(x)
        x = Flatten()(x)
        x = Dense(1, activation="sigmoid")(x)

        model = Model(inputs=input_layer, outputs=x)

        return model, embedded_model

    def createResBlocks(self, shape, n_filters):
        input_layer = Input(shape=shape)
        x = input_layer

        for i in range(3):
            x = self._resBlock(n_filters, x)

        x = Conv2D(n_filters * 2, kernel_size=(1, 1), strides=2, padding="same")(x)
        for i in range(4):
            x = self._resBlock(n_filters * 2, x)

        x = Conv2D(n_filters * 4, kernel_size=(1, 1), strides=2, padding="same")(x)
        for i in range(6):
            x = self._resBlock(n_filters * 4, x)

        x = Conv2D(n_filters * 8, kernel_size=(1, 1), strides=2, padding="same")(x)
        for i in range(3):
            x = self._resBlock(n_filters * 8, x)

        model = Model(inputs=input_layer, outputs=x)
        return model

    def _resBlock(self, n_filters, input_layer, kernel_size=(3, 3)):
        x = BatchNormalization()(input_layer)
        x = ReLU()(x)
        x = Conv2D(n_filters, kernel_size=kernel_size, padding="same")(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(n_filters, kernel_size=kernel_size, padding="same")(x)
        x = Add()([input_layer, x])
        return x

In [0]:
class DataGenerator(Sequence):

    # negative split to take from end
    def __init__(self, batch_size=50, feature_path="data_set/converted_data_set/", flip_prob=0.2, split=1):
        self.feature_path = feature_path
        self.batch_size = batch_size
        self.data_paths = self._loadFileList()
        random.shuffle(self.data_paths)

        if split > 0:
            self.data_paths = self.data_paths[:int(len(self.data_paths) * split)]
        else:
            self.data_paths = self.data_paths[int(len(self.data_paths) * split):]
        print("Using {} samples".format(len(self.data_paths)))
        self.flip_prob = flip_prob

    def __len__(self):
        return int(np.floor(len(self.data_paths) / float(self.batch_size)))

    def __getitem__(self, idx):
        paths = self.data_paths[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch = [pickle.load(open(p, "rb")) for p in paths]
        batch_y, batch_x = list(map(list, zip(*batch)))
        batch_y = np.array(batch_y)
        batch_x = np.array(batch_x)

        # random flip
        for i in range(self.batch_size):
            if random.random() < self.flip_prob:
                batch_x[i] = np.flip(batch_x[i], axis=0)
            if random.random() < self.flip_prob:
                batch_x[i] = np.flip(batch_x[i], axis=1)

        return batch_x, batch_y

    def _loadFileList(self):
        features = glob.glob(self.feature_path + "*.pickle")
        return features

In [0]:
model, embedded_model = ModelBuilder().createClassifier((256, 256, 1), 64)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 256, 256, 1)       0         
_________________________________________________________________
conv2d_290 (Conv2D)          (None, 128, 128, 64)      3200      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 64, 64, 64)        0         
_________________________________________________________________
model_17 (Model)             multiple                  22845184  
_________________________________________________________________
average_pooling2d_9 (Average (None, 4, 4, 512)         0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 8193      
Total para

In [0]:
data_generator = DataGenerator(split=1)
model_callback = ModelCheckpoint("fl_model.h5", save_best_only=True)

optimizer = Adam(lr=0.00001, decay=1e-5)
model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["accuracy"])

Using 999 samples


In [0]:
model.fit_generator(data_generator, epochs=100)

Epoch 1/100
19/19 [==============================] - 14s 729ms/step - loss: 0.0848 - acc: 0.9842
Epoch 2/100
19/19 [==============================] - 14s 722ms/step - loss: 0.0835 - acc: 0.9853
Epoch 3/100
19/19 [==============================] - 14s 724ms/step - loss: 0.0800 - acc: 0.9853
Epoch 4/100
19/19 [==============================] - 14s 726ms/step - loss: 0.0702 - acc: 0.9937
Epoch 5/100
19/19 [==============================] - 14s 729ms/step - loss: 0.0718 - acc: 0.9832
Epoch 6/100
19/19 [==============================] - 14s 728ms/step - loss: 0.0613 - acc: 0.9905
Epoch 7/100
19/19 [==============================] - 14s 725ms/step - loss: 0.0621 - acc: 0.9926
Epoch 8/100
19/19 [==============================] - 14s 726ms/step - loss: 0.0544 - acc: 0.9916
Epoch 9/100
19/19 [==============================] - 14s 727ms/step - loss: 0.0525 - acc: 0.9905
Epoch 10/100
19/19 [==============================] - 14s 726ms/step - loss: 0.0504 - acc: 0.9895
Epoch 11/100
19/19 [=========